In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical


In [3]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [6]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index)+1

print('단어 집합의 크기 : %d \n'%vocab_size)
print(t.word_index)

단어 집합의 크기 : 12 

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [12]:
sequences = list()

for line in text.split('\n') : 
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1,len(encoded)) :
        sequence = encoded[:i+1]
        sequences.append(sequence)

max_len = max(len(l) for l in sequences)
        
print('학습에 사용할 샘플의 개수 : %d \n'% len(sequences))
print('샘플의 최대 길이 : {}\n'.format(max_len))
print(sequences)

학습에 사용할 샘플의 개수 : 11 

샘플의 최대 길이 : 6

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [13]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
max_len = max(len(l) for l in sequences)


print('학습에 사용할 샘플의 개수 : %d \n'% len(sequences))
print('샘플의 최대 길이 : {}\n'.format(max_len))
print(sequences)

학습에 사용할 샘플의 개수 : 11 

샘플의 최대 길이 : 6

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [30]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print(X,"\n")
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]] 

[ 3  1  4  5  1  7  1  9 10  1 11]


In [31]:
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


# RNN 모델

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

model = Sequential()
model.add(Embedding(vocab_size,10,input_length=max_len-1))
model.add(SimpleRNN(24))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y,epochs=500, verbose=0)

In [65]:
def sentence_generation(model, t, current_word, n ) :
    init_word = current_word
    sentence = ''
    for _ in range(n) :
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        
        for word, index in t.word_index.items() :
            if index == result :
                break
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
    sentence = init_word + sentence
        
    return sentence

In [66]:
print(sentence_generation(model,t,'경마장에',5))

경마장에 있는 말이 뛰고 있다 있다
